In [137]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import struct
import os
import random
import itertools
import matplotlib.gridspec as gridspec
import tqdm
import netCDF4

In [138]:
def tostr(l):
    return "".join(i.decode() for i in l)

def read_header(f):
    satcode = tostr(struct.unpack("ccccc", f.read(5)))
    sensor  = tostr(struct.unpack("ccccc", f.read(5)))
    freq_sat = np.array(struct.unpack("f" * 13, f.read(13 * 4)))
    vangles  = np.array(struct.unpack("f" * 13, f.read(13 * 4)))
    return (satcode, sensor, freq_sat, vangles)

def read_pixel(f):
        nx, ny     = struct.unpack("ii", f.read(8))
        scntime    = np.array(struct.unpack("i" * 6, f.read(4 * 6)))
        lat, lon   = struct.unpack("ff", f.read(8))
        sfccode    = struct.unpack("i", f.read(4))
        tcwv       = struct.unpack("f", f.read(4))
        T2m        = struct.unpack("f", f.read(4))
        Tbs        = np.array(struct.unpack("f" * 13, f.read(13*4)))
        sfcprcp    = struct.unpack("f", f.read(4))
        cnvprcp    = struct.unpack("f", f.read(4))
        return (nx, ny, scntime, lat, lon, sfccode, tcwv, T2m, Tbs, sfcprcp, cnvprcp)

In [139]:
types =  [('nx', 'i4'), ('ny', 'i4')]
types += [('year', 'i4'), ('month', 'i4'), ('day', 'i4'), ('hour', 'i4'), ('minute', 'i4'), ('second', 'i4')]
types +=  [('lat', 'f4'), ('lon', 'f4')]
types += [('sfccode', 'i4'), ('tcwv', 'i4'), ('T2m', 'i4')]
types += [('Tb_{}'.format(i), 'f4') for i in range(13)]
types += [('sfcprcp', 'f4'), ('cnvprcp', 'f4')]

In [140]:
year = 14
month = 10
day = 1
year = str(year)
month = str(month)
if len (month) == 1:
    month = "0" + month
day = str(day)
if len (day) == 1:
    day = "0" + day
path = os.path.join(dendrite, "UserAreas", "Teo", "GPROFfiles", str(year) + str(month))
files = glob.glob(os.path.join(path, "GMI.CSU.20" + year + month + day + "*.dat"))

In [142]:
f = '/home/simonpf/Dendrite/UserAreas/teo/'

In [143]:
def read_multiple_files(path_list, n_rows, n_files):
    """
    Takes a list of file paths for custom .dat files
    from one given month.
    Goes through selected file:
         1: checks if ocean
         2: saves the information with some probability
    Return:
        Pixel information
    """
    
    lats = []
    lons = []
    sfccode = []
    tcwv = []
    T2m = []
    Tbs = []
    surf_precip = []
    
    for i in tqdm.tqdm(range(0,n_files)):
        # Loop over selected number of files in month folder
        path = path_list[i]
        with open(path, 'rb') as fn:
            read_header(fn) #Reads header, doesn't save anything
            for i in range(0,n_rows): #Loop over selected number of rows in given file
                try:
                    test = read_pixel(fn)
                    tmp_surf = test[5][0]
                    if tmp_surf == 1: # Makes sure that pixel is over ocean
                        lats.append(test[3])
                        lons.append(test[4])
                        sfccode.append(test[5][0])
                        tcwv.append(test[6][0])
                        T2m.append(test[7][0])
                        Tbs.append(test[8][:])
                        surf_precip.append(test[9][0])
                except:
                    break
    return (lats, lons, sfccode, tcwv, T2m, Tbs, surf_precip)

def create_training_data(n_rows_per_file, n_files_per_month, prob_to_include, day_str):
    #--------------------------------------------#
    # Wrapper function to read .dat files from
    # multiple months
    #--------------------------------------------#
    dendrite = os.path.join(os.environ["HOME"], "Dendrite")
    path = os.path.join(dendrite, "UserAreas", "Teo", "GPROFfiles")
    months = ['1409','1410','1411','1412','1501','1502','1503','1504','1505','1506','1507','1508']
    lats = []
    lons = []
    sfccode = []
    tcwv = []
    T2m = []
    Tbs = []
    surf_precip = []
    
    for i in range(0,len(months)):
        print('Starting month: ' + str(months[i]))
        files_path = os.path.join(path, months[i], '*.dat')
        print(files_path)
        files_list_tmp = np.sort(glob.glob(files_path))
        files_list = []
        
        for k in range(0,len(files_list_tmp)):
            tmp_str = files_list_tmp[k]
            tmp_test = str(months[i]) + day_str
            if tmp_test not in tmp_str:
                files_list.append(tmp_str)
                
        file_inds = np.random.choice(np.arange(0,len(files_list)),n_files_per_month)
        files_list = np.array(files_list)
        files_list_final = files_list[file_inds]
        
        lats_tmp, lons_tmp, sfccode_tmp, tcwv_tmp, T2m_tmp, Tbs_tmp, surf_precip_tmp = read_multiple_files(files_list_final,
                                                                                                           n_rows_per_file,
                                                                                                           n_files_per_month)
        lats.extend(lats_tmp)
        lons.extend(lons_tmp)
        sfccode.extend(sfccode_tmp)
        tcwv.extend(tcwv_tmp)
        T2m.extend(T2m_tmp)
        Tbs.extend(Tbs_tmp)
        surf_precip.extend(surf_precip_tmp)
    return (lats, lons, sfccode, tcwv, T2m, Tbs, surf_precip)

def create_input_and_output_arrays_ocean(Tbs, tcwv, T2m, surf_precip):
    Tbs_arr = np.array(Tbs)
    tcwv_arr = np.array(tcwv).reshape(len(tcwv),1)
    T2m_arr = np.array(T2m).reshape(len(T2m),1)

    input_array = np.concatenate((Tbs_arr,tcwv_arr), axis=1)
    input_array = np.concatenate((input_array,T2m_arr), axis = 1)
    
    output_array = np.array(surf_precip)
    return input_array, output_array

In [147]:
%rm /home/simonpf/Dendrite/UserAreas/Simon/gprof/training_data.nc

In [148]:
import os
from qprof.data import create_output_file, extract_data
dendrite = os.path.join(os.environ["HOME"], "Dendrite")
path = os.path.join(dendrite, "UserAreas", "Simon", "gprof", "training_data.nc")

days = np.arange(2, 3)
months = [9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8]
years = [14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15]

file = create_output_file(path)

for y, m in zip(years, months):
    print("Processing {}/{}".format(y, m))
    for d in tqdm.tqdm(days):
        extract_data(y, m, d, file)
file.close()






  0%|          | 0/1 [00:00<?, ?it/s]





  0%|          | 0/15 [00:00<?, ?it/s]

Processing 14/9








  7%|▋         | 1/15 [01:34<22:02, 94.48s/it]





 13%|█▎        | 2/15 [03:08<20:25, 94.25s/it]





 20%|██        | 3/15 [04:43<18:56, 94.69s/it]





 27%|██▋       | 4/15 [06:17<17:17, 94.34s/it]





 33%|███▎      | 5/15 [07:51<15:42, 94.23s/it]





 40%|████      | 6/15 [09:26<14:09, 94.43s/it]





 47%|████▋     | 7/15 [11:02<12:39, 94.95s/it]





 53%|█████▎    | 8/15 [12:36<11:03, 94.80s/it]





 60%|██████    | 9/15 [14:12<09:29, 94.92s/it]





 67%|██████▋   | 10/15 [15:52<08:02, 96.44s/it]





 73%|███████▎  | 11/15 [17:31<06:29, 97.45s/it]





 80%|████████  | 12/15 [19:13<04:55, 98.55s/it]





 87%|████████▋ | 13/15 [20:53<03:18, 99.06s/it]





 93%|█████████▎| 14/15 [22:33<01:39, 99.50s/it]





100%|██████████| 15/15 [24:16<00:00, 100.50s/it]




100%|██████████| 1/1 [24:16<00:00, 1456.65s/it]




  0%|          | 0/1 [00:00<?, ?it/s]





  0%|          | 0/13 [00:00<?, ?it/s]

Processing 14/10








  8%|▊         | 1/13 [00:39<07:50, 39.22s/it]





 15%|█▌        | 2/13 [02:24<10:49, 59.08s/it]





 23%|██▎       | 3/13 [04:10<12:10, 73.08s/it]





 31%|███       | 4/13 [05:53<12:19, 82.11s/it]





 38%|███▊      | 5/13 [07:36<11:47, 88.45s/it]





 46%|████▌     | 6/13 [09:19<10:49, 92.78s/it]





 54%|█████▍    | 7/13 [11:01<09:32, 95.46s/it]





 62%|██████▏   | 8/13 [12:44<08:08, 97.69s/it]





 69%|██████▉   | 9/13 [14:26<06:35, 98.91s/it]





 77%|███████▋  | 10/13 [16:10<05:01, 100.46s/it]





 85%|████████▍ | 11/13 [18:10<03:32, 106.43s/it]





 92%|█████████▏| 12/13 [20:08<01:49, 109.94s/it]





100%|██████████| 13/13 [22:04<00:00, 111.75s/it]




100%|██████████| 1/1 [22:04<00:00, 1324.66s/it]




  0%|          | 0/1 [00:00<?, ?it/s]





  0%|          | 0/15 [00:00<?, ?it/s]

...







  7%|▋         | 1/15 [01:35<22:15, 95.37s/it]





 13%|█▎        | 2/15 [03:10<20:40, 95.40s/it]





 20%|██        | 3/15 [04:46<19:07, 95.60s/it]





 27%|██▋       | 4/15 [06:22<17:32, 95.70s/it]





 33%|███▎      | 5/15 [07:58<15:57, 95.74s/it]





 40%|████      | 6/15 [09:33<14:19, 95.53s/it]





 47%|████▋     | 7/15 [11:08<12:43, 95.38s/it]





 53%|█████▎    | 8/15 [12:43<11:06, 95.26s/it]





 60%|██████    | 9/15 [13:40<08:21, 83.57s/it]





 67%|██████▋   | 10/15 [15:14<07:13, 86.80s/it]





 73%|███████▎  | 11/15 [16:40<05:46, 86.71s/it]





 80%|████████  | 12/15 [18:15<04:27, 89.11s/it]





 87%|████████▋ | 13/15 [19:50<03:01, 90.83s/it]





 93%|█████████▎| 14/15 [21:25<01:32, 92.18s/it]





100%|██████████| 15/15 [23:00<00:00, 93.07s/it]




100%|██████████| 1/1 [23:00<00:00, 1380.92s/it]




  0%|          | 0/1 [00:00<?, ?it/s]





  0%|          | 0/16 [00:00<?, ?it/s]

Processing 15/7








  6%|▋         | 1/16 [01:36<24:01, 96.08s/it]





 12%|█▎        | 2/16 [03:10<22:19, 95.68s/it]





 19%|█▉        | 3/16 [04:45<20:41, 95.50s/it]





 25%|██▌       | 4/16 [06:20<19:03, 95.28s/it]





 31%|███▏      | 5/16 [07:55<17:27, 95.27s/it]





 38%|███▊      | 6/16 [09:31<15:52, 95.23s/it]





 44%|████▍     | 7/16 [11:05<14:14, 94.99s/it]





 50%|█████     | 8/16 [12:39<12:38, 94.81s/it]





 56%|█████▋    | 9/16 [14:14<11:03, 94.83s/it]





 62%|██████▎   | 10/16 [15:49<09:28, 94.72s/it]





 69%|██████▉   | 11/16 [17:24<07:54, 94.81s/it]





 75%|███████▌  | 12/16 [18:59<06:19, 94.85s/it]





 81%|████████▏ | 13/16 [20:34<04:45, 95.01s/it]





 88%|████████▊ | 14/16 [22:10<03:10, 95.19s/it]





 94%|█████████▍| 15/16 [23:45<01:35, 95.09s/it]





100%|██████████| 16/16 [25:20<00:00, 95.13s/it]




100%|██████████| 1/1 [25:20<00:00, 1520.31s/it]




  0%|          | 0/1 [00:00<?, ?it/s]





  0%|          | 0/16 [00:00<?, ?it/s]

Processing 15/8








  6%|▋         | 1/16 [01:35<23:51, 95.41s/it]





 12%|█▎        | 2/16 [03:10<22:16, 95.44s/it]





 19%|█▉        | 3/16 [04:46<20:41, 95.48s/it]





 25%|██▌       | 4/16 [06:22<19:06, 95.57s/it]





 31%|███▏      | 5/16 [07:57<17:30, 95.50s/it]





 38%|███▊      | 6/16 [09:32<15:53, 95.37s/it]





 44%|████▍     | 7/16 [11:08<14:18, 95.37s/it]





 50%|█████     | 8/16 [12:42<12:41, 95.24s/it]





 56%|█████▋    | 9/16 [14:17<11:05, 95.10s/it]





 62%|██████▎   | 10/16 [15:53<09:32, 95.38s/it]





 69%|██████▉   | 11/16 [17:29<07:57, 95.45s/it]





 75%|███████▌  | 12/16 [19:04<06:21, 95.43s/it]





 81%|████████▏ | 13/16 [20:40<04:46, 95.50s/it]





 88%|████████▊ | 14/16 [22:15<03:10, 95.33s/it]





 94%|█████████▍| 15/16 [23:51<01:35, 95.51s/it]





100%|██████████| 16/16 [25:26<00:00, 95.44s/it]




100%|██████████| 1/1 [25:26<00:00, 1526.62s/it]

In [48]:
for i in range(10,20):
    print('--- STARTING FILE NBR ' + str(i+1) + ' ---')
    lats, lons, sfccode, tcwv, T2m, Tbs, surf_precip = create_training_data(1000000000,10,0.1,'01')
    x, y = create_input_and_output_arrays_ocean(Tbs,tcwv,T2m, surf_precip)
    print(x.shape)
    
    i_start = training_file.dimensions["samples"].size
    i_end = i_start + x.shape[0]
    training_file["input"][i_start : i_end, :] = x
    training_file["output"][i_start : i_end, :] = y


  0%|          | 0/10 [00:00<?, ?it/s]

--- STARTING FILE NBR 11 ---
Starting month: 1409
/home/simonpf/Dendrite/UserAreas/Teo/GPROFfiles/1409/*.dat


  0%|          | 0/10 [00:00<?, ?it/s]

Starting month: 1410
/home/simonpf/Dendrite/UserAreas/Teo/GPROFfiles/1410/*.dat


  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

Starting month: 1508
/home/simonpf/Dendrite/UserAreas/Teo/GPROFfiles/1508/*.dat


100%|██████████| 10/10 [00:03<00:00,  3.02it/s]


(4296025, 15)


IndexError: size of data array does not conform to slice

In [47]:
training_file.dimensions["samples"].size

0